# Analysis

Look into the data structure and visualise the attribution data.

In [ ]:
cd ..

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from ChannelAttribution import heuristic_models, markov_model
from src.utility import clean_data, summarise_paths

In [ ]:
df = pd.read_csv("data/attribution data.csv")
df = clean_data(df)

In [ ]:
df_agg = summarise_paths(df)

### Plot the number of conversions over time

In [ ]:
fig, ax = plt.subplots(figsize=(8, 3))

conversions = (
    df
    .query("conversion == 1")
    .groupby("date", as_index=False)
    .agg(conversions=pd.NamedAgg("conversion", "sum"))
)

ax.plot(
    conversions["date"],
    conversions["conversions"],
    marker="o",
    markersize=5,
)
ax.grid(True, alpha=.3)
plt.show()

### Apply the channel attribution modelling package

In [ ]:
df_agg.iloc[:1]

In [ ]:
# crete an attribution dataframe that is grouped by the path
df_attr = (
    df_agg.groupby(["path"], as_index=False)
    .agg(
        volume=pd.NamedAgg("path", "count"),
        conversion=pd.NamedAgg("conversion", "sum"),
        conversion_value=pd.NamedAgg("conversion_value", "sum"),
    )
)

In [ ]:
%%time
H = heuristic_models(
    Data=df_agg,
    var_path="path",
    var_conv="conversion",
    var_value="conversion_value",
)

In [ ]:
%%time
M = markov_model(
    Data=df_agg,
    var_path="path",
    var_conv="conversion",
    var_value="conversion_value",
)

In [ ]:
results = pd.merge(H, M, on="channel_name", how="inner")

In [ ]:
results.set_index("channel_name", inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

ax.bar(
    results.index,
    results
)

In [ ]:
results.filter(like="value")